In [ ]:
!pip install datasets transformers[sentencepiece]

## 获取数据

In [2]:
from transformers.utils import send_example_telemetry

send_example_telemetry("tokenizer_training_notebook", framework="none")

In [3]:
from datasets import load_dataset

In [4]:
dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [5]:
dataset

Dataset({
    features: ['text'],
    num_rows: 36718
})

In [6]:
dataset[1]

{'text': ' = Valkyria Chronicles III = \n'}

In [7]:
dataset[:5]

{'text': ['',
  ' = Valkyria Chronicles III = \n',
  '',
  ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n',
  " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making th

In [8]:
batch_size = 1000
all_texts = [dataset[i : i + batch_size]["text"] for i in range(0, len(dataset), batch_size)]

In [9]:
def batch_iterator():
    for i in range(0, len(dataset), batch_size):
        yield dataset[i : i + batch_size]["text"]

## 使用现有的Tokenizer

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [11]:
tokenizer.is_fast

True

In [12]:
new_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=25000)

In [13]:
new_tokenizer(dataset[:5]["text"])

{'input_ids': [[], [301, 8639, 9504, 3050, 301, 315], [], [4720, 74, 4825, 889, 8639, 491, 529, 672, 6944, 475, 267, 9504, 374, 2809, 529, 10879, 231, 100, 162, 255, 113, 14391, 4046, 113, 4509, 95, 18351, 4509, 256, 4046, 99, 4046, 22234, 96, 19, 264, 6437, 272, 8639, 281, 261, 3518, 2035, 491, 373, 264, 5162, 3305, 290, 344, 8639, 9504, 3050, 2616, 1822, 264, 364, 259, 14059, 1559, 340, 2393, 1527, 737, 1961, 370, 805, 3604, 288, 7577, 14, 54, 782, 337, 261, 4840, 15585, 272, 19958, 284, 1404, 1696, 284, 1822, 264, 385, 364, 261, 1431, 737, 284, 261, 8639, 906, 272, 2531, 1858, 286, 261, 1112, 9658, 281, 14059, 288, 1626, 340, 645, 6556, 344, 520, 14434, 264, 261, 1485, 3436, 7515, 290, 261, 518, 737, 288, 4750, 261, 302, 22039, 302, 264, 259, 21720, 1743, 3836, 5654, 261, 4259, 281, 4742, 490, 724, 261, 3581, 1351, 283, 1114, 579, 952, 4010, 1985, 2563, 288, 453, 2128, 807, 935, 261, 7655, 3836, 302, 2038, 314, 271, 89, 22414, 302, 272, 315], [324, 737, 1022, 1984, 284, 1525, 264, 7

In [14]:
new_tokenizer.save_pretrained("my-new-tokenizer")

('my-new-tokenizer/tokenizer_config.json',
 'my-new-tokenizer/special_tokens_map.json',
 'my-new-tokenizer/vocab.json',
 'my-new-tokenizer/merges.txt',
 'my-new-tokenizer/added_tokens.json',
 'my-new-tokenizer/tokenizer.json')

## 从0开始构建Tokenizer

### WordPiece model like BERT

In [15]:
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer

tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

In [16]:
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)

In [17]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)

In [18]:
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()

In [19]:
tokenizer.pre_tokenizer.pre_tokenize_str("This is an example!")

[('This', (0, 4)),
 ('is', (5, 7)),
 ('an', (8, 10)),
 ('example', (11, 18)),
 ('!', (18, 19))]

In [20]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

In [21]:
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

In [22]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

2 3


In [23]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", cls_token_id),
        ("[SEP]", sep_token_id),
    ],
)

In [24]:
encoding = tokenizer.encode("This is one sentence.", "With this one we have a pair.")

In [25]:
encoding.tokens

['[CLS]',
 'this',
 'is',
 'one',
 'sentence',
 '.',
 '[SEP]',
 'with',
 'this',
 'one',
 'we',
 'have',
 'a',
 'pair',
 '.',
 '[SEP]']

In [26]:
encoding.type_ids

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [27]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

In [28]:
from transformers import BertTokenizerFast

new_tokenizer = BertTokenizerFast(tokenizer_object=tokenizer)

### BPE model like GPT-2

In [29]:
tokenizer = Tokenizer(models.BPE())

In [30]:
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

In [31]:
tokenizer.pre_tokenizer.pre_tokenize_str("This is an example!")

[('This', (0, 4)),
 ('Ġis', (4, 7)),
 ('Ġan', (7, 10)),
 ('Ġexample', (10, 18)),
 ('!', (18, 19))]

In [32]:
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

In [33]:
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
tokenizer.decoder = decoders.ByteLevel()

In [34]:
from transformers import GPT2TokenizerFast

new_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)

### Unigram model like Albert

In [35]:
tokenizer = Tokenizer(models.Unigram())

In [36]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.Replace("``", '"'), normalizers.Replace("''", '"'), normalizers.Lowercase()]
)
tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()

In [37]:
tokenizer.pre_tokenizer.pre_tokenize_str("This is an example!")

[('▁This', (0, 4)), ('▁is', (4, 7)), ('▁an', (7, 10)), ('▁example!', (10, 19))]

In [38]:
trainer = trainers.UnigramTrainer(vocab_size=25000, special_tokens=["[CLS]", "[SEP]", "<unk>", "<pad>", "[MASK]"], unk_token="<unk>")
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

In [39]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")

In [40]:
tokenizer.post_processor = processors.TemplateProcessing(
    single="[CLS]:0 $A:0 [SEP]:0",
    pair="[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", cls_token_id),
        ("[SEP]", sep_token_id),
    ],
)
tokenizer.decoder = decoders.Metaspace()

In [41]:
from transformers import AlbertTokenizerFast

new_tokenizer = AlbertTokenizerFast(tokenizer_object=tokenizer)